## Preprocessing

In [69]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [77]:
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# changes made attempting to optimize / increase accuracy:
#              - bin 1 less classification
#              - make special considerations 1 column instead of 2 (1/0 instead of Y/N)
#              - bin income amount more aggressively
#              - bin some values of affiliation
#              - give a few more nodes to 2nd hidden layer

In [78]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
# also drop special considerations & status, very very few cases of these having values that matter
# application_df.drop(['EIN', 'NAME', 'SPECIAL_CONSIDERATIONS', 'STATUS'], axis = 1, inplace = True)
application_df.drop(['EIN', 'NAME'], axis = 1, inplace = True)

In [79]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# more aggressive for the optimized version
# application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17', 'T10', 'T7']
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")
    
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
class_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = [c for c in class_counts.index if c not in ['C1000', 'C2000', 'C1200', 'C3000', 'C2100', 'C7000']]

# Replace in dataframe
for c in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(c,"Other")

In [80]:
# check values of INCOME_AMT
application_df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [81]:
# bin INCOME_AMT
for i in ['1M-5M', '5M-10M']:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(i, '1M-10M')
for i in ['10M-50M', '50M+']:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(i, '10M+')
for i in ['1-9999', '10000-24999']:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(i, '1-24999')
application_df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1-24999           1271
1M-10M            1140
10M+               379
Name: INCOME_AMT, dtype: int64

In [82]:
# check values of affiliation
application_df['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [83]:
# bin affiliation
for a in ['National', 'Regional']:
    application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(a, 'Area')
application_df['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Family/Parent          64
Area                   46
Other                   4
Name: AFFILIATION, dtype: int64

In [85]:
# application_df['USE_CASE'].value_counts()

In [86]:
# bin use-case
#for u in ['CommunityServ', 'Heathcare']:
#    application_df['USE_CASE'] = application_df['USE_CASE'].replace(u, 'Other')
#application_df['USE_CASE'].value_counts()

In [87]:
#application_df['ORGANIZATION'].value_counts()

In [88]:
# bin organization
#for o in ['Co-operative', 'Corporation']:
#   application_df['ORGANIZATION'] = application_df['ORGANIZATION'].replace(o, 'Other')
#application_df['ORGANIZATION'].value_counts()

In [89]:
application_df['SPECIAL_CONSIDERATIONS'] = application_df['SPECIAL_CONSIDERATIONS'].replace('Y', 1)
application_df['SPECIAL_CONSIDERATIONS'] = application_df['SPECIAL_CONSIDERATIONS'].replace('N', 0)

In [90]:
application_df.nunique()

APPLICATION_TYPE             9
AFFILIATION                  5
CLASSIFICATION               7
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   6
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [91]:
# Convert categorical data to numeric with `pd.get_dummies`
# cat_dummies = pd.get_dummies(application_df.drop(['ASK_AMT', 'IS_SUCCESSFUL'], axis = 1))
cat_dummies = pd.get_dummies(application_df.drop(['STATUS', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL'], axis = 1))
processed_application_df = pd.concat([
    application_df[['STATUS', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL']], cat_dummies], axis = 1)
#    application_df[['ASK_AMT', 'IS_SUCCESSFUL']], cat_dummies], axis = 1)
processed_application_df.head()

,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M+,INCOME_AMT_1M-10M,INCOME_AMT_25000-99999
0,1,0,5000,1,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
1,1,0,108590,1,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
2,1,0,5000,0,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,0
3,1,0,6692,1,0,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0
4,1,0,142590,1,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0


In [92]:
# Split our preprocessed data into our features and target arrays
y = processed_application_df['IS_SUCCESSFUL']
X = processed_application_df.drop(['IS_SUCCESSFUL'], axis = 1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [93]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [94]:
pd.DataFrame(X_train_scaled).head()

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0.008818,-0.029257,-0.033233,-0.089628,-0.126792,-0.179158,0.520346,-0.220139,-0.185755,-0.191937,...,-0.653188,-0.119471,-0.03638,0.677267,0.633617,-0.194104,-0.330273,-0.102799,-0.184857,-0.348508
1,0.008818,-0.029257,-0.033233,-0.089628,-0.126792,-0.179158,0.520346,-0.220139,-0.185755,-0.191937,...,1.530954,-0.119471,-0.03638,-1.476523,0.633617,-0.194104,-0.330273,-0.102799,-0.184857,-0.348508
2,0.008818,-0.029257,-0.032191,-0.089628,-0.126792,-0.179158,0.520346,-0.220139,-0.185755,-0.191937,...,-0.653188,-0.119471,-0.03638,0.677267,-1.578240,-0.194104,-0.330273,-0.102799,-0.184857,2.869374
3,0.008818,-0.029257,-0.033233,-0.089628,-0.126792,-0.179158,0.520346,-0.220139,-0.185755,-0.191937,...,1.530954,-0.119471,-0.03638,-1.476523,0.633617,-0.194104,-0.330273,-0.102799,-0.184857,-0.348508
4,0.008818,-0.029257,-0.033233,-0.089628,-0.126792,-0.179158,0.520346,-0.220139,-0.185755,-0.191937,...,-0.653188,-0.119471,-0.03638,0.677267,0.633617,-0.194104,-0.330273,-0.102799,-0.184857,-0.348508


## Compile, Train and Evaluate the Model

In [96]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train_scaled.shape[1] # number of columns
# increasing nodes in 2nd layer
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 40

nn = Sequential()

# First hidden layer
nn.add(Dense(units = hidden_nodes_layer1, activation = 'relu', input_dim = number_input_features))

# Second hidden layer
nn.add(Dense(units = hidden_nodes_layer2, activation = 'relu'))

# Output layer
nn.add(Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 80)                3200      
                                                                 
 dense_21 (Dense)            (None, 40)                3240      
                                                                 
 dense_22 (Dense)            (None, 1)                 41        
                                                                 
Total params: 6481 (25.32 KB)
Trainable params: 6481 (25.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [97]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [98]:
# Train the model
# code from official tensorflow documentation, 2nd link in README
checkpoint_filepath = 'nn_checkpoints_optimization/weights.ckpt'
nn_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weights_only = True,
    monitor = 'val_accuracy',
    mode = 'max',
    save_freq = 5)
nn.fit(X_train_scaled, y_train, epochs = 100, callbacks = [nn_checkpoint_callback])

Epoch 1/100
804/804 [==============================] - 19s 24ms/step - loss: 0.5658 - accuracy: 0.7247
Epoch 2/100
804/804 [==============================] - 19s 24ms/step - loss: 0.5528 - accuracy: 0.7309
Epoch 3/100
804/804 [==============================] - 20s 24ms/step - loss: 0.5495 - accuracy: 0.7321
Epoch 4/100
804/804 [==============================] - 20s 25ms/step - loss: 0.5477 - accuracy: 0.7338
Epoch 5/100
804/804 [==============================] - 20s 24ms/step - loss: 0.5464 - accuracy: 0.7324
Epoch 6/100
804/804 [==============================] - 19s 24ms/step - loss: 0.5462 - accuracy: 0.7348
Epoch 7/100
804/804 [==============================] - 20s 24ms/step - loss: 0.5449 - accuracy: 0.7352
Epoch 8/100
804/804 [==============================] - 19s 24ms/step - loss: 0.5441 - accuracy: 0.7352
Epoch 9/100
804/804 [==============================] - 20s 25ms/step - loss: 0.5437 - accuracy: 0.7348
Epoch 10/100
804/804 [==============================] - 21s 26ms/step - l

KeyboardInterrupt: 

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5704 - accuracy: 0.7240 - 354ms/epoch - 1ms/step
Loss: 0.5704119801521301, Accuracy: 0.7239649891853333


In [24]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')

C:\Users\Peter Solis\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
